In [6]:
import import_ipynb
from finance_packaging import csv_agent as finance_csv_agent
from finance_packaging import sql_agent as finance_sql_agent
from inventory_packaging import csv_agent as inventory_csv_agent
from inventory_packaging import sql_agent as inventory_sql_agent
import json

In [7]:
# Initialize LLM
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [8]:
def classify_query(user_query):
    system_prompt = """
You are a router for a multi-agent data system.

Datasets: sales, finance, inventory, spend
Sources: csv, sql

Return your answer strictly in JSON:
{
  "dataset": "<sales|finance|inventory|spend>",
  "source": "<csv|sql>"
}
"""

    full_prompt = f"{system_prompt}\n\nUser Query: {user_query}\n\nYour Response:"

    response = llm.predict(full_prompt).strip()

    try:
        result = json.loads(response)
        dataset = result.get("dataset", "").lower()
        source = result.get("source", "").lower()
        return dataset, source
    except:
        print("Invalid router response:", response)
        return None, None

In [9]:
def master_agent(user_query):
    dataset, source = classify_query(user_query)

    if dataset == "finance" and source == "csv":
        return finance_csv_agent.run(user_query)
    elif dataset == "finance" and source == "sql":
        return finance_sql_agent.run(user_query)

    elif dataset == "inventory" and source == "csv":
        return inventory_csv_agent.run(user_query)
    elif dataset == "inventory" and source == "sql":
        return inventory_sql_agent.run(user_query)

    else:
        return "❌ Unable to classify query."

In [10]:
while True:
    query = input("\nAsk your question (or type 'exit'): ")
    if query.lower() == "exit":
        break
    response = master_agent(query)
    print(f"\n🧠 Response:\n{response}")


Invalid router response: ```json
{
  "dataset": "finance",
  "source": "sql"
}
```

🧠 Response:
❌ Unable to classify query.
